In [37]:
!pip install fastapi uvicorn nest-asyncio pyngrok
!pip install passlib python-jose sqlalchemy fastapi uvicorn
!pip install python-multipart



In [38]:
import os

SECRET_KEY = os.getenv("SECRET_KEY", "default_secret_key")
print(SECRET_KEY)


default_secret_key


In [39]:
%%writefile main.py

from fastapi import FastAPI, Depends, HTTPException, Form
from fastapi.security import OAuth2PasswordBearer, OAuth2PasswordRequestForm
from pydantic import BaseModel
from sqlalchemy import create_engine, Column, Integer, String, Boolean
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, Session
from passlib.context import CryptContext
from jose import jwt
from datetime import datetime, timedelta

# Configuration
SECRET_KEY = "default_secret_key"
ALGORITHM = "HS256"
ACCESS_TOKEN_EXPIRE_MINUTES = 30
DATABASE_URL = "sqlite:///./test.db"

# Database Setup
engine = create_engine(DATABASE_URL, connect_args={"check_same_thread": False})
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

def get_password_hash(password: str):
    return pwd_context.hash(password)

def verify_password(plain_password: str, hashed_password: str):
    return pwd_context.verify(plain_password, hashed_password)

def create_access_token(data: dict):
    to_encode = data.copy()
    expire = datetime.utcnow() + timedelta(minutes=ACCESS_TOKEN_EXPIRE_MINUTES)
    to_encode.update({"exp": expire})
    return jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)

oauth2_scheme = OAuth2PasswordBearer(tokenUrl="token")

class User(Base):
    __tablename__ = "users"
    id = Column(Integer, primary_key=True, index=True)
    username = Column(String, unique=True, index=True)
    email = Column(String, unique=True, index=True)
    hashed_password = Column(String)
    is_active = Column(Boolean, default=True)

Base.metadata.create_all(bind=engine)

class UserIn(BaseModel):
    username: str
    email: str
    password: str

class UserOut(BaseModel):
    username: str
    email: str

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

def get_current_user(token: str = Depends(oauth2_scheme), db: Session = Depends(get_db)):
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        username = payload.get("sub")
        if username is None:
            raise HTTPException(status_code=401, detail="Invalid authentication credentials")
        user = db.query(User).filter(User.username == username).first()
        if user is None:
            raise HTTPException(status_code=401, detail="Invalid authentication credentials")
        return user
    except jwt.JWTError:
        raise HTTPException(status_code=401, detail="Invalid authentication credentials")

app = FastAPI()

@app.post("/register", response_model=UserOut)
def register(user: UserIn, db: Session = Depends(get_db)):
    db_user = db.query(User).filter((User.username == user.username) | (User.email == user.email)).first()
    if db_user:
        raise HTTPException(status_code=400, detail="Username or email already registered")
    hashed_password = get_password_hash(user.password)
    new_user = User(username=user.username, email=user.email, hashed_password=hashed_password)
    db.add(new_user)
    db.commit()
    db.refresh(new_user)
    return new_user

@app.post("/token")
def login(form_data: OAuth2PasswordRequestForm = Depends(), db: Session = Depends(get_db)):
    user = db.query(User).filter(User.username == form_data.username).first()
    if not user or not verify_password(form_data.password, user.hashed_password):
        raise HTTPException(status_code=400, detail="Invalid username or password")
    access_token = create_access_token(data={"sub": user.username})
    return {"access_token": access_token, "token_type": "bearer"}

@app.get("/profile", response_model=UserOut)
def profile(current_user: User = Depends(get_current_user)):
    return current_user


Overwriting main.py


In [40]:
!ngrok config add-authtoken 2otPD1PnMnVXgtxAfOkn9mt9FEG_5aVffF6NZe2zokkU8ssbi


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [41]:
from fastapi import FastAPI

app = FastAPI()

@app.get("/some_path")
async def read_item():
    return {"message": "This is some path!"}


In [42]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

# Allow nested event loops
nest_asyncio.apply()

# Expose the app to the internet
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")




Public URL: NgrokTunnel: "https://eb13-34-55-179-145.ngrok-free.app" -> "http://localhost:8000"


In [43]:
# Start the FastAPI app
uvicorn.run("main:app", host="0.0.0.0", port=8000)

INFO:     Started server process [571]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     2407:aa80:116:86c0:41e7:d50:ccc3:860:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     2407:aa80:116:86c0:41e7:d50:ccc3:860:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2407:aa80:116:86c0:41e7:d50:ccc3:860:0 - "GET / HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [571]
